In [1]:
script("tarballs");

In [15]:
@b = unpack_tarball("fano-v3d.tgz");
#$polytope = load("fano-v3d/F.3D.0000.poly");
#@b = unpack_tarball("fano-v7d-7.tgz");
#$polytope = load("fano-v7d-7/fano.pnew.7D.070000.poly")

In [3]:
application "fan";

In [12]:
sub Reverse {
# argument: Rational Matrix
    return transform(simplex($_[0]->rows() - 1), $_[0])->get_attachment("REVERSE_TRANSFORMATION");
}

sub IntegerMatrix {
# argument: Rational Matrix
# checks if all elements are integer
    $rows = $_[0]->rows();
    $cols = $_[0]->cols();
    
    $fmat = convert_to<Float>($_[0]);
    for (my $row = 0; $row < $rows; $row += 1) {
        for (my $col = 0; $col < $cols; $col += 1) {
            $elem = $fmat->elem($row, $col);
            if (not ($elem == int $elem)) {
                return 0;
            }
        }
    }
    return 1;
}
sub Bilateral {
#argument: Integer matrix of Rays (strings)
#checks bilaterality relative to standard simplex
#does not check if all vertices are different
    $rows = $_[0]->rows();
    $cols = $_[0]->cols();
    
    for (my $row = 0; $row < $rows; $row += 1) {
        $neg_orth = 1;
        $basis = 1;
        $summa = 0;
        for (my $col = 0; $col < $cols; $col += 1) {
            $elem = $_[0]->elem($row, $col);
            $summa += $elem;
            if ($elem < 0 or $elem > 1 or $summa > 1) {$basis = 0;}
            if ($elem > 0)  {$neg_orth = 0;}
        }
        if ($summa == 0 or not ($neg_orth or $basis)) {return 0;}
    }
    return 1;
}
sub SpecialFacet {
#argument: Integer matrix of Vertices (strings)
#checks that standard simplex is a special facet
#does not check, that standard simplex is a facet
    $rows = $_[0]->rows();
    $cols = $_[0]->cols();
    
    for (my $col = 0; $col < $cols; $col += 1) {
        $summa = 0;
        for (my $row = 0; $row < $rows; $row += 1) {
            $elem = $_[0]->elem($row, $col);
            $summa += $elem;
        }
        if ($summa < 0) {return 0;}
    }
    return 1;
}

In [17]:
$pol_number = 0;
print "P\tnV\tnF\tILI\tSp\tBil\tAnd\tXor\tEq\n"; #Pconc
#for ($pol_number = 7522; $pol_number < 7622; $pol_number += 1) {
for ($pol_number = 72156; $pol_number < 72256; $pol_number += 1) {
#foreach my $polytope (@b) {
#    $polytope = load("fano-v6d/F.6D.$pol_number.poly");
    $polytope = load("fano-v7d-7/fano.pnew.7D.0$pol_number.poly");
    
    $p = polarize($polytope);
    $nfacets = $p->N_FACETS;
    $nvertices = $p->N_VERTICES;
    $facets = $p->VERTICES_IN_FACETS;
    $vertices = $p->VERTICES;
    #print "Polytope $pol_number\nVertices\n", rows_labeled($vertices), "\n"; # Polytopes/info
    #print "Special Facets: ", $p->SPECIAL_FACETS, "\n\n"; #Polytopes
    
    $ILI = 0;
    $Sp = 0;
    $Bil = 0;
    $And = 0;
    $Xor = 0;
    $Eq = $nfacets;
    
    @special = ();
    @ili = ();
    @bilateral = ();
    
    $incidence = dense($facets);
    for (my $facet = 0; $facet < $nfacets; $facet += 1) {
        @f = ();
        for (my $vertex = 0; $vertex < $nvertices; $vertex += 1) {
            if ($incidence->elem($facet, $vertex)) {
                push(@f, $vertex);
            }
        }
        $facet_set = new Set(@f);
        
        $matrix = $vertices->minor($facet_set, ~[0]);
        $rmatrix = Reverse($matrix);
        $transformed = $vertices->minor(All, ~[0]) * $rmatrix;
        #print "Facet ", $facets->row($facet), " to basis:\n$transformed", "\nby matrix\n", transpose($rmatrix), "\n"; # info
        
        $li = IntegerMatrix($rmatrix) * IntegerMatrix($matrix);
        $sp = SpecialFacet($transformed);
        $bil = Bilateral($transformed);
        
        $ILI += $li;
        $Sp += $sp;
        $Bil += $bil;
        $And += $sp * $bil;
        $Xor += ($sp + $bil) % 2;
        $Eq -= ($sp + $bil) % 2;
        
        push(@special, $sp);
        push(@ili, $li);
        push(@bilateral, $bil);
    }
    print "$pol_number\t$nvertices\t$nfacets\t$ILI\t$Sp\t$Bil\t$And\t$Xor\t$Eq\n"; # Pconc
    #print "ILI\tSp\tBil\tFacet\n"; # Polytopes
    
    for (my $facet = 0; $facet < $nfacets; $facet += 1) {
        #print "$ili[$facet]\t$special[$facet]\t$bilateral[$facet]\t", $facets->row($facet), "\n"; # Polytopes
    }
    #print "\n\n"; #Polytopes/info
    
    #$pol_number += 1;
}

P	nV	nF	ILI	Sp	Bil	And	Xor	Eq
72156	11	54	54	18	9	0	27	27
72157	11	54	54	36	0	0	36	18
72158	10	36	36	18	0	0	18	18
72159	12	72	72	24	12	0	36	36
72160	11	54	54	24	6	0	30	24
72161	11	54	54	18	9	0	27	27
72162	10	36	36	18	3	0	21	15
72163	10	36	36	24	2	0	26	10
72164	13	96	96	48	48	0	96	0
72165	12	72	72	48	24	0	72	0
72166	12	72	72	36	36	0	72	0
72167	12	72	72	72	0	0	72	0
72168	11	48	48	18	6	0	24	24
72169	11	48	48	36	12	0	48	0
72170	11	48	48	24	24	0	48	0
72171	11	48	48	12	12	0	24	24
72172	11	48	48	16	8	0	24	24
72173	11	48	48	24	24	0	48	0
72174	11	48	48	48	0	0	48	0
72175	10	30	30	12	3	0	15	15
72176	11	54	54	36	18	0	54	0
72177	10	30	30	12	3	0	15	15
72178	10	30	30	24	6	0	30	0
72179	10	30	30	15	15	0	30	0
72180	10	30	30	10	5	0	15	15
72181	10	30	30	10	5	0	15	15
72182	10	30	30	5	10	0	15	15
72183	11	48	48	24	0	0	24	24
72184	11	54	54	27	27	0	54	0
72185	11	54	54	54	0	0	54	0
72186	10	30	30	15	15	0	30	0
72187	10	30	30	10	5	0	15	15
72188	10	30	30	20	5	0	25	5
72189	10	30	30	15	15	0	30	0
72190	10	30	30	30	0	0